In [1]:
# Config Data Structure
import IPython
import IPython.display

import pandas as pd
import numpy as np
import pprint as pp
import random as ran
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import random as ran
import math

from functools import reduce
from collections import Counter
from datetime import datetime as dt
from pymongo import MongoClient as mc
from functools import reduce

sns.set(rc={'figure.figsize': (15.7, 13.27)})
plt.rcParams['figure.figsize'] = 15.7,13.27
plt.rcParams['font.family'] = 'AppleGothic'

mongo_uri = "mongodb://localhost:27017"
client = mc(mongo_uri)
keti_db = client.keti_pattern_recognition

household_col = keti_db.household_info
cluster_col = keti_db.cluster_info
weather_col = keti_db.weather_info

In [2]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               train_df, val_df, test_df,
               label_columns=None):
    # Store the raw data.
    self.train_df = train_df
    self.val_df = val_df
    self.test_df = test_df

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(train_df.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

def plot(self, model=None, plot_col='kw (15min)', max_subplots=3):
  inputs, labels = self.example
  plt.figure(figsize=(12, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(max_subplots, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]')
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend().remove()

  plt.xlabel('Time [h]')

WindowGenerator.plot = plot

def make_dataset(self, data):
  data = np.array(data, dtype=np.float32)
  ds = tf.keras.preprocessing.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=1,
      shuffle=True,
      batch_size=32,)

  ds = ds.map(self.split_window)

  return ds

WindowGenerator.make_dataset = make_dataset

@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def val(self):
  return self.make_dataset(self.val_df)

@property
def test(self):
  return self.make_dataset(self.test_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.test))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.val = val
WindowGenerator.test = test
WindowGenerator.example = example

MAX_EPOCHS = 40

def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience,
                                                    mode='min')

  model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError()])

  history = model.fit(window.train, epochs=MAX_EPOCHS,
                      validation_data=window.val,
                      callbacks=[early_stopping])
  return history

In [3]:
# DB Config
uid = "아파트4-103-106"

# TimeSlot In
hh_db_datas = household_col.find_one({"uid": uid})
hh_db_datas

hh_datas = pd.DataFrame(columns=['Date Time','kw (15min)'])
timeslot = hh_db_datas['timeslot']

wt_db_datas = weather_col.find()
wt_datas = pd.DataFrame()

hh_datas['kw (15min)'] = [ts['power'] for ts in timeslot]
hh_datas['Date Time'] = [ts['time'] for ts in timeslot]

date_time = pd.to_datetime(hh_datas.pop('Date Time'), format="%Y-%m-%d T%H:%M Z")

hh_datas.index = date_time
hh_datas

,kw (15min)
Date Time,
2018-05-01 00:00:00,0.099
2018-05-01 00:15:00,0.103
2018-05-01 00:30:00,0.084
2018-05-01 00:45:00,0.079
2018-05-01 01:00:00,0.105
...,...
2019-04-30 22:45:00,0.030
2019-04-30 23:00:00,0.033
2019-04-30 23:15:00,0.038


In [4]:
# Split Data Code
split_datas = round(hh_datas[0::4], 3)

# Merging Data Code
count = 0
merge_datas = pd.DataFrame(columns=['kw (15min)'])
merge_datas['kw (15min)'] = [
    round(hh_datas[
        idx * 4:
        (idx + 1) * 4
    ].mean().values[0], 3)
    for idx in range(0, round(len(hh_datas) / 4))
]
merge_datas.index = [
    hh_datas.index[idx * 4]
    for idx in range(0, round(len(hh_datas) / 4))
]

In [5]:
# training datas 80%
# validation datas 10%
# validation datas 10%
val_days = []
test_days = []

val_data_percentage = 10
test_data_percentage = 10

# Mon.1 ~ Mon.12
for month in range(1,13):
    val_day_list = []
    test_day_list = []
    day_list = list(set(
        split_datas[split_datas.index.month == month].index.day
    ))
    month_datas = split_datas[split_datas.index.month == month].copy()
    start_day = day_list[0]
    end_day = day_list[len(day_list) - 1] + 1
    
    # 10 percentage split
    val_length = round(len(day_list)) / val_data_percentage
    test_length = round(len(day_list)) / test_data_percentage
    
    while True:
        if (len(val_day_list) >= val_length) and \
            (len (test_day_list) >= test_length):
            break
        while True:
            val_day = ran.randrange(start_day, end_day)
            test_day = ran.randrange(start_day, end_day)

            if (val_day != test_day) and \
                (val_day not in val_day_list + test_day_list) and \
                (test_day not in test_day_list + val_day_list):
                val_days.extend(
                    month_datas[
                        month_datas.index.day == val_day
                    ].index
                )
                test_days.extend(
                    month_datas[
                        month_datas.index.day == test_day
                    ].index
                )
                val_day_list.append(val_day)
                test_day_list.append(test_day)
                break;

train_days = split_datas.drop((val_days + test_days)).index

In [74]:
# Set Split Datas
train_split_df = split_datas.loc[train_days].copy()
val_split_df = split_datas.loc[val_days].copy()
test_split_df = split_datas.loc[test_days].copy()

# Set Merge Datas
train_merge_df = merge_datas.loc[train_days].copy()
val_merge_df = merge_datas.loc[val_days].copy()
test_merge_df = merge_datas.loc[test_days].copy()

In [75]:
def get_season(month):
    if month in [3,4,5]:
        return "봄"
    elif month in [6,7,8]:
        return "여름"
    elif month in [9,10,11]:
        return "가을"
    else:
        return "겨울"
    
train_mean = train_split_df.mean()
train_std = train_split_df.std()
print("split data // mean: {}, std: {}".format(train_mean.values, train_std.values))

norm_split = dict({
    "all": (split_datas - train_mean) / train_std,
    "train": (train_split_df - train_mean) / train_std,
    "val": (val_split_df - train_mean) / train_std,
    "test": (test_split_df - train_mean) / train_std,
})
norm_split['season_bak'] = dict({
    'train': pd.DataFrame(),
    'val': pd.DataFrame(),
    'test': pd.DataFrame()
})
norm_split['season_bak']['train']['season'] = [get_season(month) for month in norm_split['train'].index.month]
norm_split['season_bak']['train'].index = norm_split['train'].index

norm_split['season_bak']['val']['season'] = [get_season(month) for month in norm_split['val'].index.month]
norm_split['season_bak']['val'].index = norm_split['val'].index

norm_split['season_bak']['test']['season'] = [get_season(month) for month in norm_split['test'].index.month]
norm_split['season_bak']['test'].index = norm_split['test'].index

train_mean = train_merge_df.mean()
train_std = train_merge_df.std()
print("merge data // mean: {}, std: {}".format(train_mean.values, train_std.values))

norm_merge = dict({
    "all": (merge_datas - train_mean) / train_std,
    "train": (train_merge_df - train_mean) / train_std,
    "val": (val_merge_df - train_mean) / train_std,
    "test": (test_merge_df - train_mean) / train_std
})


split data // mean: [0.09726165], std: [0.10081229]
merge data // mean: [0.09729734], std: [0.08417301]


In [76]:
windows = dict()
seasons = ['봄', '여름', '가을', '겨울']

for season in seasons:
    season_windows = dict()
    season_idx = dict({
        'train': norm_split['season_bak']['train'][norm_split['season_bak']['train']['season'] == season].index,
        'val': norm_split['season_bak']['val'][norm_split['season_bak']['val']['season'] == season].index,
        'test': norm_split['season_bak']['test'][norm_split['season_bak']['test']['season'] == season].index,
    })
    for WINDOW_WIDTH in range(3, 21):
        season_windows['{} hours'.format(WINDOW_WIDTH)] = WindowGenerator(
            input_width=WINDOW_WIDTH,
            label_width=1,
            shift=1,
            label_columns=['kw (15min)'],
            train_df=norm_split['train'].loc[season_idx['train']],
            val_df=norm_split['val'].loc[season_idx['val']],
            test_df=norm_split['test'].loc[season_idx['test']]
        )
    windows[season] = season_windows

windows

{'봄': {'3 hours': Total window size: 4
  Input indices: [0 1 2]
  Label indices: [3]
  Label column name(s): ['kw (15min)'],
  '4 hours': Total window size: 5
  Input indices: [0 1 2 3]
  Label indices: [4]
  Label column name(s): ['kw (15min)'],
  '5 hours': Total window size: 6
  Input indices: [0 1 2 3 4]
  Label indices: [5]
  Label column name(s): ['kw (15min)'],
  '6 hours': Total window size: 7
  Input indices: [0 1 2 3 4 5]
  Label indices: [6]
  Label column name(s): ['kw (15min)'],
  '7 hours': Total window size: 8
  Input indices: [0 1 2 3 4 5 6]
  Label indices: [7]
  Label column name(s): ['kw (15min)'],
  '8 hours': Total window size: 9
  Input indices: [0 1 2 3 4 5 6 7]
  Label indices: [8]
  Label column name(s): ['kw (15min)'],
  '9 hours': Total window size: 10
  Input indices: [0 1 2 3 4 5 6 7 8]
  Label indices: [9]
  Label column name(s): ['kw (15min)'],
  '10 hours': Total window size: 11
  Input indices: [0 1 2 3 4 5 6 7 8 9]
  Label indices: [10]
  Label column 

In [79]:
val_performance = dict()
performance = dict()
models = dict()

for season in seasons:
    season_val_performance = dict()
    season_performance = dict()
    season_models = dict()
    for WINDOW_WIDTH in range(3, 21):
        IPython.display.clear_output()
        key = "{} hours".format(WINDOW_WIDTH)
        print("##### {} training start #####".format(key))

        cur_window = windows[season][key]
        lstm_model = tf.keras.models.Sequential([
            # Shape [batch, time, features] => [batch, time, lstm_units]
            tf.keras.layers.LSTM(64, return_sequences=True, activation="tanh"),
            # Shape => [batch, time, features]
            tf.keras.layers.Dense(units=1)
        ])

        compile_and_fit(lstm_model, cur_window)
        season_val_performance[key] = lstm_model.evaluate(
                    cur_window.val
                )
        season_performance[key] = lstm_model.evaluate(
                    cur_window.test
                )
        season_models[key] = lstm_model
    val_performance[season] = season_val_performance
    performance[season] = season_performance
    models[season] = season_models

##### 20 hours training start #####
Epoch 1/40
51/51 [==============================] - 3s 36ms/step - loss: 1.0321 - mean_absolute_error: 0.6071 - val_loss: 1.1285 - val_mean_absolute_error: 0.6692
Epoch 2/40
51/51 [==============================] - 0s 8ms/step - loss: 0.9946 - mean_absolute_error: 0.5920 - val_loss: 1.1119 - val_mean_absolute_error: 0.6732
Epoch 3/40
51/51 [==============================] - 0s 8ms/step - loss: 0.9856 - mean_absolute_error: 0.5932 - val_loss: 1.1008 - val_mean_absolute_error: 0.6765
Epoch 4/40
51/51 [==============================] - 0s 7ms/step - loss: 0.9820 - mean_absolute_error: 0.5945 - val_loss: 1.0992 - val_mean_absolute_error: 0.6674
Epoch 5/40
51/51 [==============================] - 0s 8ms/step - loss: 0.9802 - mean_absolute_error: 0.5879 - val_loss: 1.1037 - val_mean_absolute_error: 0.6739
Epoch 6/40
51/51 [==============================] - 0s 8ms/step - loss: 0.9784 - mean_absolute_error: 0.5841 - val_loss: 1.0965 - val_mean_absolute_error